# MobileNet

In [1]:
import tensorflow as tf
import keras
from keras.layers import Input, Conv2D, BatchNormalization, Activation, DepthwiseConv2D, Dense, GlobalAveragePooling2D
from keras.models import Model

## The Model

In [2]:
def Conv(X_input, filters, kernel_size, strides = (1,1), padding = 'same'):
    X = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = padding)(X_input)
    X = BatchNormalization()(X)
    return Activation('relu')(X)

In [3]:
def DepConv(X_input, filters, strides = (1,1)):
    X = DepthwiseConv2D(kernel_size = (3,3), strides = strides, padding = 'same')(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    return Conv(X, filters, (1,1))

In [4]:
X_input = Input(shape=(224,224,3))

X = Conv(X_input, 32, (3,3), (2,2))

X = DepConv(X, 64)
X = DepConv(X, 128, (2,2))

X = DepConv(X, 128)
X = DepConv(X, 256, (2,2))

X = DepConv(X, 256)
X = DepConv(X, 512, (2,2))

for i in range(5):
    X = DepConv(X, 512)

X = DepConv(X, 1024, (2,2))
X = DepConv(X, 1024)

X = GlobalAveragePooling2D()(X)
X = Dense(units=1000, activation='softmax')(X)
    
mobilenet = Model(inputs = X_input, outputs = X)
mobilenet.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 32)     

In [5]:
mobilenet.save("MobileNet.keras")

## Model Training

In [11]:
from keras.preprocessing.image import ImageDataGenerator
bs = 64

train_gen = ImageDataGenerator().flow_from_directory('../input/imagenetmini-1000/imagenet-mini/train/', 
                                                    target_size=(224,224), 
                                                    batch_size=bs, 
                                                    class_mode='categorical')

val_gen = ImageDataGenerator().flow_from_directory('../input/imagenetmini-1000/imagenet-mini/val/', 
                                                    target_size=(224,224), 
                                                    batch_size=bs, 
                                                    class_mode='categorical')

Found 34745 images belonging to 1000 classes.
Found 3923 images belonging to 1000 classes.


In [7]:
mobilenet.compile(loss='categorical_crossentropy', 
                 optimizer='adam', 
                 metrics=['accuracy'])

In [ ]:
from livelossplot import PlotLossesKeras

mobilenet.fit_generator(train_gen, 
                       epochs=5, 
                       steps_per_epoch=34745//bs, 
                       validation_data=val_gen, 
                       validation_steps=3923//bs,
                       callbacks=[PlotLossesKeras()], 
                       verbose=1)

Epoch 1/5
